In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from imageio import imread
from collections import OrderedDict
import zbar
import copy
import csv
from io import StringIO

from skimage.color import rgb2gray
from skimage.transform import estimate_transform, warp_coords, warp, rescale
from skimage.exposure import equalize_adapthist, equalize_hist
from skimage.morphology import closing, opening
from skimage.filters import gaussian
from scipy.signal import argrelextrema
import numpy as np
from scipy.ndimage import convolve
from ats.main import POSITIONS, EDGE_POSITIONS

import base64
import zlib
import json

In [ ]:
dina4 = np.array([21, 29.7])

In [ ]:
img = imread("./IMG_0356.JPG")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(rescale(img, 1))

In [ ]:
gray_img = (255*gaussian(rgb2gray(img), 0.1)).astype(np.uint8)

In [ ]:
scanner = zbar.Scanner()
results = scanner.scan(gray_img)
print(len(results))
splited_data = {}
qr_positions = {}
for result in results:
    pos, data = result.data.decode('utf8').split(":")
    splited_data[pos] = data
    qr_positions[pos] = result.position
    print(result.type, result.data, result.quality, result.position)

In [ ]:
list(splited_data.keys())

In [ ]:
data = ""
for pos in POSITIONS:
    data += splited_data[pos]

In [ ]:
x = base64.b85decode(data)
x = zlib.decompress(x)
x = json.loads(x.decode('utf8'))
config = x
config

In [ ]:
def coords2rect(coords):
    coords = np.array(coords)
    xy = coords.min(axis=0)
    width = np.max(coords[:, 0] - xy[0])
    height = np.max(coords[:, 1] - xy[1])
    return xy, width, height


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(img)

centers = {}
for name, coords in qr_positions.items():
    centers[name] = np.mean(coords, axis=0)
    rect = Rectangle(*coords2rect(coords), fill=False, color='green')
    ax.add_patch(rect)
    
ax.scatter([c[0] for c in centers.values()], [c[1] for c in centers.values()], c='red')

In [ ]:
# C = A X
# C X^-1 = A I

C = np.zeros((len(EDGE_POSITIONS), 2))
for i, pos in enumerate(EDGE_POSITIONS):
    C[i] = centers[pos]
C.shape

In [ ]:
pos = x['positions']
qr_width = 3
qh = qr_width / 2
pos

In [ ]:
X = np.array([    
    [pos['left'] + qh, -pos['top'] + qh],
    [pos['right'] + qh, -pos['top'] + qh],
    [pos['left'] + qh, -pos['bottom'] + qh],
    [pos['right'] + qh, -pos['bottom'] + qh],
])
X

In [ ]:
transform = estimate_transform('projective', X, C)

In [ ]:
transform.params

In [ ]:
x = np.arange(0, 29.7)
y = np.arange(0, 21.001)
x, y = np.meshgrid(x, y)
coords = np.stack([x, y], axis=-1).reshape(-1, 2)
coords

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(img)

t_coords = transform(coords)

ax.scatter(t_coords[:, 0], t_coords[:, 1], c='red')


In [ ]:
dpcm = 180 / 2.54
X_dp = X * dpcm

def cm2dots(cm):
    return cm * dpcm

transform = estimate_transform('projective', X_dp, C)

In [ ]:
(2*dina4 * dpcm).astype(np.int)

In [ ]:
img_warp = warp(img, inverse_map=transform, output_shape=(dina4 * dpcm).astype(np.int).T)

In [ ]:
img_warp.shape

In [ ]:
fig, ax = plt.subplots(figsize=(18, 12))
ax.imshow(img_warp)

In [ ]:
img_equal = equalize_hist(rgb2gray(img_warp))

In [ ]:
img_equal = equalize_adapthist(rgb2gray(img_warp), kernel_size=2*np.array([dpcm, dpcm]))

In [ ]:
s = 0.3
fig, ax = plt.subplots(figsize=(18*s, 12*s))
ax.imshow(img_equal, cmap='gray')

In [ ]:
img_binary = img_equal > 0.6

In [ ]:
fig, ax = plt.subplots(figsize=(18*s, 12*s))
ax.imshow(img_binary, cmap='gray')

In [ ]:
img_closed = closing(img_binary, np.ones((int(dpcm*0.15), int(dpcm*0.1))))
fig, ax = plt.subplots(figsize=(18*s, 12*s))
ax.imshow(img_closed, cmap='gray')

In [ ]:
def build_h_kernel():
    s = int(dpcm * 0.25)
    n = 5
    h_kernel = []
    for i in range(n):
        h_kernel.extend([1] * s + [-1] * s)

    return np.array([h_kernel])
    
h_kernel = build_h_kernel()
fig, ax = plt.subplots(figsize=(18, 12))
img_kernel = img_closed.copy()
for i in range(30):
    img_kernel[90+i, 200:200+len(h_kernel[0])] = 0.5*h_kernel.flatten() + 0.5

ax.imshow(img_kernel, cmap='gray')

In [ ]:
img_to_conv = img_closed.copy()
img_to_conv[200:220, :] = 0
img_conv_h = convolve(2*img_to_conv.astype(np.float)-1, h_kernel)

fig, ax = plt.subplots(figsize=(18, 12))
ax.imshow(img_conv_h, cmap='gray')

In [ ]:
h_line_power = np.abs(img_conv_h).mean(axis=1) / len(h_kernel.flatten())
plt.plot(h_line_power)
plt.plot(0.3*(h_line_power > 0.3))

In [ ]:
def get_first_edge_lines(img, line_number_and_power, threshold=0.2, mode='vertical'):
    first_match = None
    edges = []
    line_indicies = []
    for i, power in line_number_and_power:
        if power > threshold:
            if first_match is None:
                first_match = i
            
            if mode == 'vertical':
                line = img[:, i]
            elif mode == 'horizontal':
                line = img[i, :]
            else:
                raise Exception()
                
            edges.append(convolve(line, [1, 1, -1, -1]))
            line_indicies.append(i)
        elif first_match is not None:
            break
    return np.array(edges), np.array(line_indicies)

h_edges, h_lines = get_first_edge_lines(img_equal, reversed(list(enumerate(h_line_power))), mode='horizontal')
h_edges.shape

In [ ]:
fig, ax = plt.subplots(figsize=(18, 2))
#for edge in edges:
    # ax.plot(edge)
ax.plot(np.mean(h_edges, axis=0))
plt.show()

In [ ]:
def get_grid(edges, offset=None, threshold=0.5):
    if offset is None:
        offset = 0
    edges = edges[offset:]
    begins = edges < - threshold
    ends = edges > threshold
    
    local_extrema, = argrelextrema(edges, np.greater)
    local_minima, = argrelextrema(edges, np.less)
    
    begins = offset + local_extrema[edges[local_extrema] > threshold]
    ends = offset + local_minima[edges[local_minima] < - threshold]
    grid = np.concatenate([begins, ends])
    grid.sort()
    return grid

In [ ]:
h_grid = get_grid(np.mean(h_edges, axis=0), offset=100)

In [ ]:
def build_v_kernel():
    s = int(dpcm * 0.45)
    n = 5
    v_kernel = []
    for i in range(n):
        v_kernel.extend([1] * s + [-1] * s)
    return np.array(v_kernel).reshape(-1, 1)
v_kernel = build_v_kernel()
fig, ax = plt.subplots(figsize=(18*s, 12*s))
img_kernel = img_closed.copy()
off = 145
for i in range(30):
    img_kernel[off:off+len(v_kernel), -i] = 0.5*v_kernel.flatten() + 0.5

ax.imshow(img_kernel, cmap='gray')

In [ ]:
img_conv_v = convolve(2*img_closed.astype(np.float) - 1, v_kernel) 

fig, ax = plt.subplots(figsize=(18*s, 12*s))
ax.imshow(img_conv_v, cmap='gray')

In [ ]:
plt.plot(np.abs(img_conv_v).mean(axis=0) / len(v_kernel.flatten())) 

In [ ]:
v_power_per_line = np.abs(img_conv_v).mean(axis=0) / len(v_kernel.flatten())
plt.plot(v_power_per_line)
plt.plot(0.3*(v_power_per_line > 0.3))

In [ ]:
v_edges, v_lines = get_first_edge_lines(img_equal, reversed(list(enumerate(v_power_per_line))), mode='vertical')

In [ ]:
fig, ax = plt.subplots(figsize=(18, 2))
#for edge in edges:
    # ax.plot(edge)
ax.plot(np.mean(v_edges, axis=0))
plt.show()

In [ ]:
v_grid = get_grid(np.mean(v_edges, axis=0))

In [ ]:
def filter_v_grid(grid, max_derivation=cm2dots(0.1)):
    def in_bounds(dist):
        return dist is not None and median - max_derivation < dist < median + max_derivation 
    
    distances = grid[1:] - grid[:-1]
    
    median = np.median(distances)
    keep = set()
    remove = set()
    
    for i, pos in enumerate(grid):
        dist_next = grid[i+1] - pos if i + 1 < len(grid) else None
        dist_prev = pos - grid[i - 1]  if i - 1 >= 0 else None
        dist = dist_next or dist_prev
        if in_bounds(dist_next) or in_bounds(dist_prev):
            keep.add(i)
        else:
            remove.update(range(i))
            
            print(len(keep))
            if len(keep) < len(config['names']) + 2:
                keep.clear()
            else:
                break
    return grid[list(keep)]
v_filtered_grid = filter_v_grid(v_grid)

In [ ]:
s = 1.4
fig, ax = plt.subplots(figsize=(18*s, 12*s))
ax.imshow(img_equal, cmap='gray')


ax.vlines(h_grid, cm2dots(3), cm2dots(17), colors='b')
ax.hlines(v_filtered_grid, cm2dots(3), cm2dots(28), colors='r')
plt.show()

In [ ]:
def get_centers(grid):
    return grid[:-1] + (grid[1:] - grid[:-1]) // 2

In [ ]:
v_centers = get_centers(v_filtered_grid)
h_centers = get_centers(h_grid)

In [ ]:
s = 1.4
fig, ax = plt.subplots(figsize=(18*s, 12*s))
ax.imshow(img_equal, cmap='gray')


ax.vlines(h_centers, cm2dots(3), cm2dots(17), colors='b')
ax.hlines(v_centers, cm2dots(3), cm2dots(28), colors='r')
plt.show()

In [ ]:
list(config.keys())

In [ ]:
img_marked_closed = closing(img_binary, np.ones((int(dpcm*0.08), int(dpcm*0.1))))
fig, ax = plt.subplots(figsize=(18*s, 12*s))
ax.imshow(img_marked_closed, cmap='gray')

In [ ]:
def get_pixel_pos(i, j):
    return v_centers[i], h_centers[j]   

def get_patch(i, j, border=1):
    vi, vj = [i], h_centers[j]
    hb = v_filtered_grid[i] + border
    he = v_filtered_grid[i+1] - border
    wb = h_grid[j] + border
    we = h_grid[j+1] - border
    patch = img_marked_closed[hb:he, wb:we]
    return patch

def is_marked(patch, black_threshold=0.1, min_percent_black=0.6):
    if type(patch) == tuple:
        patch = get_patch(patch[0], patch[1])
    percentage_black = np.sum(patch < black_threshold) / patch.size
    return percentage_black > min_percent_black


In [ ]:
marked = []
for i in range(len(v_centers)-1):
    for j in range(len(h_centers)-1):
        if is_marked(get_patch(i, j)):
            marked.append(get_pixel_pos(i, j))
marked = np.array(marked)

In [ ]:
s = 1.4
fig, ax = plt.subplots(figsize=(18*s, 12*s))
ax.imshow(img_equal, cmap='gray')

plt.scatter(marked[:, 1], marked[:, 0], c='#00ff00', marker='X', s=80)
plt.show()

In [ ]:
patch = get_patch(1, 40)
#patch = get_patch(11, 0)
patch = get_patch(9, 95, border=0)

plt.imshow(patch, cmap=plt.cm.gray, vmin=0, vmax=1)
np.sum(patch < 0.25) / np.sum(patch < 10)


In [ ]:
plt.hist(patch.flatten(), bins=20)

In [ ]:
box_offset = 0
names = config['names']
items = config['items']
consumed_per_person = OrderedDict([
    (n, OrderedDict([(item['name'], 0) for item in items]))
    for n in names])
costs_per_person = copy.deepcopy(consumed_per_person)
for item in config['items']:
    boxes = item['boxes']
    for i in range(box_offset, box_offset + boxes):
        for j, person in enumerate(config['names']):
            if is_marked((j, i)):
                consumed_per_person[person][item['name']] += 1
                costs_per_person[person][item['name']] += item['price']
    box_offset += boxes
    

In [ ]:
consumed_per_person
costs_per_person

In [ ]:
for person, costs in costs_per_person.items():
    print("{:13} |  {:0.2f}".format(person, sum(costs.values())))

In [ ]:
buf = StringIO()
writer = csv.writer(buf)
writer.writerow(['name'] + [item['name'] for item in items])

In [ ]:
for person, cost in costs_per_person.items():
    writer.writerow([person] + list(cost.values()))

In [ ]:
print(buf.getvalue())